In [5]:
import pandas as pd

# Load the dataset
df = pd.read_csv("data/personality_dataset.csv")

# Display basic information about the dataset
print(df.info())
print(df.head())
print(df.describe(include='all'))

# Check for missing values
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           2837 non-null   float64
 1   Stage_fear                 2827 non-null   object 
 2   Social_event_attendance    2838 non-null   float64
 3   Going_outside              2834 non-null   float64
 4   Drained_after_socializing  2848 non-null   object 
 5   Friends_circle_size        2823 non-null   float64
 6   Post_frequency             2835 non-null   float64
 7   Personality                2900 non-null   object 
dtypes: float64(5), object(3)
memory usage: 181.4+ KB
None
   Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
0               4.0         No                      4.0            6.0   
1               9.0        Yes                      0.0            0.0   
2               9.0        Yes                      1

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Separate target variable
X = df.drop("Personality", axis=1)
y = df["Personality"]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["float64", "int64"]).columns

# Preprocessing pipelines for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Create a preprocessor object using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ])

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Original X shape: {X.shape}")
print(f"Processed X shape: {X_processed.shape}")
print(f"Original y shape: {y.shape}")
print(f"Encoded y shape: {y_encoded.shape}")
print("Sample of encoded y:", y_encoded[:5])
print("Sample of processed X (first 5 rows, first 5 columns):\n", X_processed[:5, :5])

Original X shape: (2900, 7)
Processed X shape: (2900, 9)
Original y shape: (2900,)
Encoded y shape: (2900,)
Sample of encoded y: [0 1 1 0 0]
Sample of processed X (first 5 rows, first 5 columns):
 [[ 4.  4.  6. 13.  5.]
 [ 9.  0.  0.  0.  3.]
 [ 9.  1.  2.  5.  2.]
 [ 0.  6.  7. 14.  8.]
 [ 3.  9.  4.  8.  5.]]


In [7]:
pip install scikit-learn


     ---------------------------------------- 10.7/10.7 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 MB 4.5 MB/s eta 0:00:00
  Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)
import numpy as np

# Define the preprocessor (update according to your pipeline)
# Example: Assuming 'preprocessor' is already defined as part of your pipeline
# preprocessor = ... 

# Hyperparameter grid for RandomForestClassifier
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False],
}

# Create the pipeline with preprocessing and classifier
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available cores
    verbose=2
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Best hyperparameters and the corresponding model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:", best_params)

# Evaluate the model on the training set
y_train_pred = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

print("\nTraining Set Evaluation:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-Score: {train_f1:.4f}")

# Evaluate the model on the test set
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')
test_conf_matrix = confusion_matrix(y_test, y_test_pred)

print("\nTesting Set Evaluation:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print("Confusion Matrix:\n", test_conf_matrix)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Hyperparameters: {'classifier__bootstrap': True, 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 100}

Training Set Evaluation:
Accuracy: 0.9366
Precision: 0.9368
Recall: 0.9366
F1-Score: 0.9366

Testing Set Evaluation:
Accuracy: 0.9293
Precision: 0.9296
Recall: 0.9293
F1-Score: 0.9293
Confusion Matrix:
 [[278  24]
 [ 17 261]]


In [ ]:
"""from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Create the full pipeline with preprocessing and model
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

# Make predictions
y_pred = model_pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)"""

Accuracy: 0.9224
Precision: 0.9088
Recall: 0.9317
F1-Score: 0.9201
Confusion Matrix:
 [[276  26]
 [ 19 259]]


In [17]:
# Example of making a prediction on new data
# Let's create a sample new data point (ensure it has the same columns as the training data)
new_data = pd.DataFrame([{
    'Time_spent_Alone': 5,
    'Stage_fear': 'No',
    'Social_event_attendance': 3,
    'Going_outside': 7,
    'Drained_after_socializing': 'No',
    'Friends_circle_size': 10,
    'Post_frequency': 6
}])

# Make prediction
new_prediction_encoded = best_model.predict(new_data)
new_prediction_personality = label_encoder.inverse_transform(new_prediction_encoded)

print(f"New data prediction: {new_prediction_personality[0]}")

New data prediction: Extrovert
